In [6]:
import os, csv, pickle, gzip, torch
import numpy as np
import ruptures as rpt
import seaborn as sns
import pandas as pd
from scipy.signal import savgol_filter

import matplotlib.pyplot as plt
from matplotlib.text import Text
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
from matplotlib.widgets import Button

In [7]:
def read_gzip(path):    
    # All VERSIONs should match (pickle, python, pandas)
    output = []
    assert os.path.exists(path), 'ReadGzip Assertion: path is null: {0}'.format(path)
    with gzip.open(path, 'rb') as file:
        print("open gzip")
        output = pickle.load(file)     
    return output


#==========LOAD================================
spath = "data/data_feature/sorted-params.csv"
sfile = open(spath, 'r', encoding='utf-8')
sfile = list(csv.reader(sfile))

# ver2 file makes vector feature velocity to hold (t) and (t-1)value instead of simple subtraction that is useless
# file_name = "0919-f18r16ver2"
file_name = "1017-f18r16ver3"
fpath = "data/data_feature/" + file_name + ".gz"
ffile = read_gzip(fpath)

dfpath = "data/data_feature/0216-df.csv"
dffile = pd.read_csv(dfpath, na_values=" NaN").dropna(how='all', axis=1)
dffile = dffile[['sub_idx','file_idx','length', 'is_exclude', 'h','v','d','x','y','z']]
# print(dffile.head)


"""
dffile: idx: 0-15: uniform setup
dffile: idx: 16-23: nonuniform setup
"""
params_df = dffile[['h','v','d']]
params_tensor = torch.tensor(params_df.values, dtype=torch.float32)
PARAMS = params_tensor.reshape(24, 144, 3)




open gzip


In [8]:
def PlotTrimFiles(inputList, argList, titlename='default', isSave=False):
    '''
    plot 2D list of abs sum of columns
    get interactive trimList and return it
    can also apply savgol filter
    return: 3 List of breakPoints, minPoints, TrimPoints
    '''
    breakPointList =[]
    minIndexList =[]    
    StartFrameList = []
    EndFrameList = []
    DelFileList = []

    # filePath = os.path.join(os.getcwd(), 'Data', 'png', titlename) 
    # try:
    #     os.makedirs(filePath, exist_ok=True)
    # except:
    #     None

    for fileIdx in range(len(inputList)):
        arr = inputList[fileIdx]
        if argList is not None:
            arr = arr[:, argList] #
        frames = arr.shape[0]
        fig, ax = plt.subplots()
        x = np.arange(frames)
        y = np.sum(np.abs(arr.reshape(arr.shape[0], -1)), axis=1) #sum all elements
        y = savgol_filter(y, 21, 3)
        

        # find breakpoints in plot
        algo = rpt.Pelt(model="rbf").fit(y)
        breakPoints = algo.predict(pen=10)
        breakPointList.append(breakPoints)


        # for each breakpoint, find local min
        localMinXList =[]
        localMinYList = []        
        for idx in range(len(breakPoints)-1):
            values = y[breakPoints[idx]: breakPoints[idx+1]]
            minIndex = np.argmin(values) + breakPoints[idx]
            localMinXList.append(minIndex)
            localMinYList.append(y[minIndex])
            plt.text(minIndex, y[minIndex], '_' + str(minIndex),  size=15)   
        localMinXList = np.array(localMinXList)
        minIndexList.append(localMinXList)


        # Plot (Marker, exclude label) to fill list
        title = "file-sum: " + str(fileIdx)
        ylabel = 'abs-sum'
        ax.plot(x, y, linewidth=1, label=ylabel, color='b')
        ax.scatter(localMinXList,  localMinYList, 100, picker=True)
        ax.set_xlabel('Click to Exclude', picker=True, bbox=dict(facecolor='red'))
        plt.title(titlename + '_' + title)        
        plt.grid()
        plt.legend()  
        
        # pick (scatter, or exclude text)
        def onPick(event):
            if isinstance(event.artist, Text):
                text = event.artist #'PickEvent' object has no attribute 'ind'
                print("exclude file: ", fileIdx, text.get_text())
                EndFrameList.append(-1)
                plt.close()
            else:
                index = event.ind
                print("(file, index, finalFrameIdx): ", fileIdx, index, localMinXList[index])
                EndFrameList.append(localMinXList[index][0])
                plt.close()

        # 'pick_event' picks index for clicked artist in canvas
        fig.canvas.mpl_connect('pick_event', onPick)    
        plt.show()
        plt.close() # close twice to avoid error

    return EndFrameList

In [ ]:
velTs = fClass.listFeaturesVelTs #[f][frame,feature,16]    
fClass.l_trim_idx = fvis.PlotTrimFiles(velTs, l_v, titlename='practice-velTs', isSave=False)